In [2]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.grid_search import RandomizedSearchCV

In [3]:
df = pd.read_csv("./work_dir/my_midterm_train_split.csv")

In [4]:
y=df['y']
X=df.drop(['y'], axis=1)

In [5]:
X.shape

(120000, 67)

In [6]:
y.shape

(120000,)

In [7]:
#setup our ML Pipe
scaler = RobustScaler()
logit = LogisticRegression(n_jobs=4)
pipe=Pipeline([('scaler', scaler), ('logit',logit)])

In [8]:
#setup grid search
hyperparameters = { 'logit__C':np.arange(.01, 10, .1)}
search = RandomizedSearchCV(pipe, hyperparameters, cv=5, scoring='roc_auc')

In [9]:
search.fit(X,y)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=Pipeline(steps=[('scaler', RobustScaler(copy=True, with_centering=True, with_scaling=True)), ('logit', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=4,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'logit__C': array([ 0.01,  0.11, ...,  9.81,  9.91])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring='roc_auc', verbose=0)

In [10]:
search.best_params_

{'logit__C': 9.1099999999999994}

In [11]:
search.best_score_

0.68240420069507957

In [12]:
## Make Prediction to get on the board

In [18]:
kaggle_test = pd.read_csv("./work_dir/my_midterm_kaggle_test.csv")
#selected_features = pd.read_csv("./work_dir/feature_support.csv")

In [22]:
kaggle_test_selected = kaggle_test.ix[:, selected_features['0'].values]

In [26]:
prediction = pd.DataFrame(search.best_estimator_.predict_proba(kaggle_test_selected)[:,1])

In [27]:
prediction.columns=['y']

In [29]:
prediction.to_csv("logit_model_prediction.csv", index_label="Id")